# fase 5 Encontrar posicion de la historia laboral

In [ ]:
from pathlib import Path
import shutil
import unicodedata

def normalize_name(s: str) -> str:
    """
    Convierte a minúsculas, elimina tildes, '_'→espacio y colapsa espacios repetidos.
    """
    s = s.strip().lower().replace('_', ' ')
    s = ''.join(
        c for c in unicodedata.normalize('NFKD', s)
        if not unicodedata.combining(c)
    )
    return ' '.join(s.split())

def build_control_map(control_dir: Path, suffix: str = '_hoja_control') -> dict:
    """
    Crea un diccionario { nombre_normalizado -> Path(al Excel) }.
    """
    mapping = {}
    for xlsx in control_dir.glob(f'*{suffix}*.xlsx'):
        stem = xlsx.stem
        person = stem[:-len(suffix)] if stem.lower().endswith(suffix) else stem
        key = normalize_name(person)
        mapping[key] = xlsx
    return mapping

def distribute_and_overwrite_excels(control_map: dict, pdf_root: Path):
    """
    Recorre pdf_root buscando PDFs; si coincide el nombre, copia el Excel junto al PDF,
    **sobrescribiendo** cualquier archivo con el mismo nombre.
    """
    for pdf in pdf_root.rglob('*.pdf'):
        name = normalize_name(pdf.stem)
        excel = control_map.get(name)
        if excel:
            dst = pdf.parent / excel.name
            shutil.copy2(excel, dst)
            print(f'↻ Sobrescrito {excel.name} → {pdf.parent}')
        else:
            print(f'⚠ Sin control para {pdf.name}')

# ——— Aquí defines tus rutas ———
control_dir = Path(r"C:\Users\juans\Downloads\dev_prev\formato final\salida")    # carpeta con *_hoja_control.xlsx
pdf_root    = Path(r"C:\Users\juans\Documents\resource\historias\_01_historias")     # carpeta raíz donde buscar PDFs


# ——— Ejecutar ———
control_map = build_control_map(control_dir)
print(f"> Encontradas {len(control_map)} hojas de control en '{control_dir}'\n")
distribute_and_overwrite_excels(control_map, pdf_root)